In [1]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool as ThreadPool
import math
import numpy as np
import random
import copy


In [2]:
def loadData(file_name):
    data = read_csv(os.getcwd()+'/'+file_name, sep=',',header=1,index_col=False)
    return data

In [3]:
def okrugl(x, do_chisla):
    return (x + do_chisla - x % do_chisla) if do_chisla - x % do_chisla < x % do_chisla  else (x - (x % do_chisla))

In [4]:
class Box():
    #конструктор создает коробку с SKU и начальными параметрами. Изначальная ротация = 3
    #okrugl - радиус округления в меньшую сторону по дельте
    def __init__(self, SKU, length, width, height, weight, strength, aisle, caustic, okrugl_d):
        self.SKU = SKU
        self.size = np.array([okrugl(length,okrugl_d),okrugl(width, okrugl_d),okrugl(height,okrugl_d)])
        self.coords = self.size
        self.rotation = 3
        self.weight = weight
        self.strength = strength
        self.aisle = aisle
        self.caustic = caustic
        self.attPoint = np.array([-1,-1,-1])
        self.farCorner = np.array([-1,-1,-1])
        
    #private функция возвращает матрицу поворота 1-6
    def __getRotationMatrix(self, rotation):
        if rotation == 1:
            return np.array([[1,0,0],[0,0,1],[0,1,0]])
        elif rotation == 2:
            return np.array([[0,0,1],[1,0,0],[0,1,0]])
        elif rotation == 3:
            return np.array([[1,0,0],[0,1,0],[0,0,1]])
        elif rotation == 4:
            return np.array([[0,0,1],[0,1,0],[1,0,0]])
        elif rotation == 5:
            return np.array([[0,1,0],[1,0,0],[0,0,1]])
        elif rotation == 6:
            return np.array([[0,1,0],[0,0,1],[1,0,0]])
    
    #метод позволяющий повернуть коробку в положение 1-6 (3 положение по умолчанию)
    def setRotation(self, rotation):
        self.rotation = rotation
        self.coords = np.dot(self.size, self.__getRotationMatrix(rotation))
        self.farCorner = self.attPoint+self.coords
    
    #вернуть реальные координаты в свой СО
    def getCoords(self):
        return self.coords
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU
    
    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]
    
    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]
    
    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)
    
    #вернуть свое текущее положение
    def getRotation(self):
        return self.rotation
    
    def getWeigth(self):
        return self.weight
    
    def getStrength(self):
        return self.strength
    
    def getAisle(self):
        return self.aisle
    
    def getCaustic(self):
        return self.caustic
    
    def __repr__(self):
        return 'Box(SKU=%s, rot=%s, coords=%s, attPoint=%s, farCorner=%s)' % (self.SKU, self.rotation, self.coords,self.attPoint, self.farCorner)
    
    def displace(self, dcoords):
        self.attPoint = np.array(dcoords)
        self.farCorner = self.attPoint+self.coords
        
    def getAttPoints(self):
        p1 = self.attPoint+self.coords*[1,0,0]
        p2 = self.attPoint+self.coords*[0,1,0]
        p3 = self.attPoint+self.coords*[1,1,0]
        p4 = self.attPoint+self.coords*[0,0,1]
        x = list([list(p1),list(p2),list(p3),list(p4)])
        return x
    
    def getAttPoint(self):
        return list(self.attPoint)
    
    def getFarCorner(self):
        return list(self.farCorner)
    
    def getBottomCenterMass(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0 , (self.farCorner[1]+self.attPoint[1]) / 2.0, self.attPoint[2])
    
    def getMaxZ(self):
        return self.farCorner[2]
    
    def getVolume(self):
        return self.size[0]*self.size[1]*self.size[2]
    
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))
    
    def getPointsToCheck(self):
        #не использовать, вместо него проверка центра масс
        return ((self.attPoint[0], self.attPoint[1], self.attPoint[2]),
                (self.attPoint[0], self.farCorner[1], self.attPoint[2]),
                (self.farCorner[0], self.farCorner[1], self.attPoint[2]),
                (self.farCorner[0], self.attPoint[1], self.attPoint[2]))
    
    def getAnswer(self):
        return [self.SKU, 
                self.attPoint[0], self.attPoint[1], self.attPoint[2], 
                self.farCorner[0], self.farCorner[1], self.farCorner[2],
                self.aisle,
                self.weight
               ]
    
    def __eq__(self, other):
        """Override the default Equals behavior"""
        if (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False
    
    def __ne__(self, other):
        if  not (self.getSKU() == other.getSKU() and
           self.getRotation() == other.getRotation() and
           self.getAttPoint() == other.getAttPoint() and
           self.getSize() == other.getSize()):
            return True
        else:
            return False

In [5]:
def getSortedAttPoints(attpoints):
    x = attpoints[:]
    def point_is_better(p1, p2):
        def getDistance(point):
            return math.sqrt(point[0] ** 2 + point[1] ** 2 + point[2] ** 2)

        def evristics(p1,p2):
            if p1[0]<p2[0]:
                return 1
            else:
                return -1

        if p1[2] < p2[2]:
            return 1
        elif p1[2] > p2[2]:
            return -1
        elif getDistance(p2)>getDistance(p1):
            return 1
        elif getDistance(p1)>getDistance(p2):
            return -1
        else:
            return evristics(p1,p2)
    x.sort(cmp=point_is_better, reverse=True)
    return x
    

In [6]:
#Проверка на пересечение объемов
def theyMerge(b1,b2):
    def getOverlap(a, b):
        return max(0, min(a[1], b[1]) - max(a[0], b[0]))

    def plane1(b1,b2):
        l_x_1 = (b1.attPoint[0], b1.farCorner[0])
        l_x_2 = (b2.attPoint[0], b2.farCorner[0])
        l_z_1 = (b1.attPoint[2], b1.farCorner[2])
        l_z_2 = (b2.attPoint[2], b2.farCorner[2])

        return getOverlap(l_x_1, l_x_2)*getOverlap(l_z_1, l_z_2)

    def plane2(b1,b2):
        l_x_1 = (b1.attPoint[0], b1.farCorner[0])
        l_x_2 = (b2.attPoint[0], b2.farCorner[0])
        l_y_1 = (b1.attPoint[1], b1.farCorner[1])
        l_y_2 = (b2.attPoint[1], b2.farCorner[1])
        #print l_x_1, l_x_2
        #print l_y_1, l_y_2
        return getOverlap(l_x_1, l_x_2)*getOverlap(l_y_1, l_y_2)

    def plane3(b1,b2):
        l_z_1 = (b1.attPoint[2], b1.farCorner[2])
        l_z_2 = (b2.attPoint[2], b2.farCorner[2])
        l_y_1 = (b1.attPoint[1], b1.farCorner[1])
        l_y_2 = (b2.attPoint[1], b2.farCorner[1])
        #print l_z_1, l_z_2
        #print l_y_1, l_y_2
        #tuple sintead of list, mb faster
        return getOverlap(l_z_1, l_z_2)*getOverlap(l_y_1, l_y_2)
    
    if plane1(b1,b2)*plane2(b1,b2) > 0 or plane2(b1,b2)*plane3(b1,b2) > 0 or plane1(b1,b2)*plane3(b1,b2)>0:
        return True
    else:
        return False

In [7]:
def getInitialList(data, ok):
    ls = []
    for each in range(len(data)):
        count = data["Quantity"][each]
        while count > 0:
            ls.append(Box(data['SKU'][each], data['Length'][each], data['Width'][each], data['Height'][each], data['Weight'][each],data['Strength'][each],data['Aisle'][each],data['Caustic'][each],ok))
            count-=1
    return ls

In [8]:

def pointBelongs(point, plane):
    if (point[2] == plane[0][2] and
        point[0] <= plane[1][0] and
        point[0] >= plane[0][0] and
        point[1] <= plane[1][1] and
        point[1] >= plane[0][1]):
        return True
    else:
        return False  

In [9]:
def getMaxZ(chromosome):
        max_z = chromosome[0].farCorner[1] #максимальная высота первого блока дальней точки
        for each in chromosome[1:]:
            if each.getMaxZ()>max_z:
                max_z = each.getMaxZ()
        return max_z

In [10]:
def getPercolation(chromosome):
        sum_volume = 0.
        for each in chromosome:
            sum_volume += each.getVolume()
            
        max_volume = getMaxZ(chromosome) * pallet_size[0] * pallet_size[1]
        return sum_volume / max_volume

In [11]:
def fitness_function(chromosome):
    return getPercolation(chromosome)  

In [12]:
def decode(chromosome):
    #изначально только 0 0 0 точка
    attPoints = [[0,0,0]]
    chromosome[0].displace(attPoints.pop(0)) #ставим первый элемент в угол, удаляем точку
    attPoints += chromosome[0].getAttPoints() #добавляем точки присоединения
    attPoints = getSortedAttPoints(attPoints) #сортируем согласно правилам точки
    status = [0, 0, 0]
    
    for i in range(1,len(chromosome)):
        displaced = False
        for p_num, point in enumerate(attPoints):
        #берем следующую точку и блок, ставим его и проверяем что:
            chromosome[i].displace(point)

            #центр масс не свисает
            
            if chromosome[i].getAttPoint()[2] == 0: #ака лежит на полу то все ок:
                itDoesntHang = True
            else:
                for j in range(i):
                    itDoesntHang = False
                    if pointBelongs(chromosome[i].getBottomCenterMass(), chromosome[j].getUpperPlane()):
                        itDoesntHang = True
                        break;

            #не выходит за границы данной плоскости палеты [0] (x) < pallet_x [3] && (z) < pallet_z
            if chromosome[i].farCorner[0] <= pallet_size[0] and chromosome[i].farCorner[1] <= pallet_size[1]:
                itFits = True
            else:
                itFits = False

            #блоки не налагаются друг на друга в объеме
            theyDontMerge = True
            for j in range(i):
                if theyMerge(chromosome[i], chromosome[j]):
                    theyDontMerge = False
                    break

            if theyDontMerge and itFits and itDoesntHang:
                #print 'YAY'
                attPoints.pop(p_num)
                displaced = True
                attPoints += [x for x in chromosome[i].getAttPoints() if x not in attPoints] #добавляем точки присоединения
                attPoints = getSortedAttPoints(attPoints) #сортируем согласно правилам точки
                break
        if not displaced: #Houston we have problems here
            return None
    return chromosome#success
            #print(attPoints)

In [13]:
def shuffle(chromosome, beta):
    x = copy.deepcopy(chromosome)
    for i in range(len(chromosome)):
        x.append(x.pop(random.randint(0, len(chromosome)-1)))
        if random.random() < beta:
            x[random.randint(0, len(chromosome)-1)].setRotation(random.randint(1,6))
    return x

In [14]:
#функция для сортировки популяции
def compare(chr1, chr2):
    return int((fitness_function(chr2)*1000-fitness_function(chr1)*1000))

In [236]:
def crossover(chromosome1, chromosome2):
    point = random.randint(1,len(chromosome1)-2)
    print point
    kid1 = copy.deepcopy(chromosome1[:point])
    kid3 = copy.deepcopy(chromosome1[point:])
    to_add = copy.deepcopy(chromosome2[:])
    for each in kid1:
        for other in to_add:
            if each.getSKU() == other.getSKU():
                kid3.append(other)
                to_add.remove(other)
                break
    kid1 += to_add
    
    kid2 = copy.deepcopy(chromosome2[:point])
    kid4 = copy.deepcopy(chromosome2[point:])
    to_add = copy.deepcopy(chromosome1[:])
    for each in kid2:
        for other in to_add:
            if each.getSKU() == other.getSKU():
                kid4.append(other)
                to_add.remove(other)
                break
    kid2 += to_add 
    
    return [kid1, kid2, kid3, kid4]
        

to_add = copy.deepcopy(chromosome2[:])
    kid2 = []
    for each in kid1:
        for other in to_add:
            if each.getSKU() == other.getSKU():
                kid2.append(other)
                to_add.remove(other)
                break
    kid1 += to_add
    kid2 += copy.deepcopy(chromosome1[point:])
    
    kid3 = copy.deepcopy(chromosome1[point:])
    to_add = copy.deepcopy(chromosome2[:])
    kid4 = []
    for each in kid3:
        for other in to_add:
            if each.getSKU() == other.getSKU():
                kid4.append(other)
                to_add.remove(other)
                break
    kid3 += to_add
    kid4 += copy.deepcopy(chromosome1[point:])

def mutation(chromosome, alpha):
    x = copy.deepcopy(chromosome[:])
    prob = random.random()
    if prob < alpha:
        x[random.randint(0, len(chromosome)-1)].setRotation(random.randint(1,6))
    return x

In [264]:
def mutation(chromosome):
    x = copy.deepcopy(chromosome[:])
    for i in range(1, random.randint(1,random.randint(3,len(x)-1))):
        x[random.randint(0,len(chromosome)-1)].setRotation(random.randint(1,6))
    return x

In [17]:
def getNewCHR(ok):
    initialList = getInitialList(data,ok)
    temp = shuffle(initialList, initial_mutation)
    return temp

In [376]:
##### настройки
pallet_size = (1200,800) #z,x: y - верх.
population_size = 7
initial_mutation = 0.3
max_deep = 15

In [377]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['5.csv', 'COS-TEST-1.csv']

In [378]:
file_ = '5.csv'
percolations = []

In [379]:
for okrugl_x in [1,2,5,10]:
    print "Файл #","(",file_,")"
    data = loadData(file_)
    print(data)
    #делаем начальную популяцию без клонов
    population = []
    
    while len(population) < population_size:
        new_population = []
        while len(new_population) < population_size:
            new_population.append(getNewCHR(okrugl_x))
        pool = ThreadPool(4)
        ans = list(pool.map(decode, new_population))
        pool.close()
        pool.join()

        for i, done in enumerate(ans):
            if done and len(population) < population_size and done not in population:
                population.append(ans[i])

    population.sort(cmp=compare)
    print "начальная популяция:"
    for each in population:
        print fitness_function(each), len(each)
    
    count = 0
    while count < max_deep and fitness_function(population[0]) < 0.9:
        count += 1
        new_population = []
        for i in range(1,population_size):
            new_population += crossover(population[0], population[i])

        rand_int1 = random.randint(1,len(population)-1)
        rand_int2 = random.randint(1,len(population)-1)
        while rand_int1 == rand_int2:
            rand_int1 = random.randint(0,len(population)-1)
            rand_int2 = random.randint(0,len(population)-1)

        new_population += crossover(population[rand_int1], population[rand_int2])
        rand_int1 = random.randint(1,len(population)-1)
        new_population.append(mutation(population[rand_int1]))
        rand_int1 = random.randint(1,len(population)-1)
        new_population.append(shuffle(population[rand_int1],initial_mutation))

        for x in range(2):
            new_population.append(getNewCHR(okrugl_x))

        pool = ThreadPool(4)
        ans = list(pool.map(decode, new_population))
        pool.close()
        pool.join()

        population += [x for x in ans if x and x not in population]

        population.sort(cmp = compare)

        population = population[:population_size]
        print "поколение", count,"best =", fitness_function(population[0])
        
    print "лучшее", fitness_function(population[0])
    percolations.append(fitness_function(population[0]))
    chromosome = population[0]
    
    packed_items = []
    total_weight = 0
    for each in chromosome:
        packed_items.append(each.getAnswer())
        total_weight += each.getWeigth()
    answer = DataFrame(packed_items)
    answer.columns = ['SKU','x_1^i','y_1^i','z_1^i','x_2^i','y_2^i','z_2^i','Aisle','Weight']
    print "solution"
    print getMaxZ(chromosome), getPercolation(chromosome), total_weight
    print answer
    print "\n\n\n"
    f = open(os.getcwd()+'/ANSWERS/'+'answer'+'okrugl_'+str(okrugl_x)+file_, 'wb')
    f.write(str(getMaxZ(chromosome)) + ' ' + str(getPercolation(chromosome)) + ' ' + str(total_weight) + '\n')
    answer.to_csv(f,index = False)
    f.close()

Файл # ( 5.csv )
        SKU  Quantity  Length  Width  Height  Weight  Strength  Aisle  Caustic
0    600595         1     457    302     115   12333         4      7        0
1    600598         1     455    303     115   12273         4      7        0
2    601472         1     290    252     224    2687         3      7        0
3    700125         1     414    273      80    7020         4      7        0
4    700392         1     307    230     114    6340         4      7        0
5    700986         1     350    260     109    8540         4      6        0
6    701616         1     323    239     160    9100         4      6        0
7    702241         1     385    256      69    4233         4      7        0
8    703307         1     381    185     172    4793         3      6        0
9    703675         1     304    137     175    2380         3      6        0
10   704014         1     307    226     231   11680         4      7        0
11   704025         1     290    17

In [380]:
percolations

[0.6159051737776445,
 0.5847785332314744,
 0.6076195759368836,
 0.6022356719367589]

In [381]:
average = sum(percolations) / len(percolations)
average

0.6026347387206904